In [1]:
# default_exp urban_municipal_analytics

# Urban Municipal Pricing Analytics

> API details.

In [8]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script

In [328]:
#export
import pandas as pd
from fastcore.script import *

In [34]:
#export 
def open_file(path:str):
    lines = None
    try:
        with open(path) as file:
            lines = file.readlines()
    except Exception as e:
        print(f"unable to open file:{path}")  
        print(f"the following error occcurred: {e}")      
    return lines

In [297]:
#export
class UrbanMunicipalMarketPrices:
    def __init__(self,date:str,company:str,headers:list,data,locations:list):
        self.company = company
        self.date = date
        self.headers = headers
        self.data = data 
        self.locations = locations
    def __repr__(self):
        x = {
            
             "header":self.headers,
             "data" : self.data[0:10],
             
          
          }
        return f"{x}"    



In [367]:
#export
class  UrbanMunicipalMarketPricesFactory:

    
    def __init__(self,data_list:list):
        self.date = (self.__get_date(data_list)).lower().replace(",","_").replace(" ","_").replace("__","_")
        self.company= self.__get_company(data_list)
        self.headers = self.__get_headers(data_list)
        self.locations = self.__get_locations(data_list)
        self.data = self.__get_data(data_list)
        u = UrbanMunicipalMarketPrices(self.date,self.company,self.headers[:13],self.data,self.locations)
        

    def __get_date(self,data_list):
        date = (data_list[1]
        .replace("Week","")
        .replace("Ending","")
        .replace("|","")
        .replace("\n","")
        .replace('"',"").lstrip())
        return date 
    def __get_company(self,data_list):
        company = data_list[2].replace("\n","")
        return company
    def __get_headers(self,data_list):
        headers = data_list[9].split("|")
        headers = list(map(lambda s:s.replace("\n",''), headers))
        
        headers[:0] = ["market"]
        headers[:0] = ["date"]
        return headers

    def __get_data(self,data_list):
        headers = data_list[9].split("|")
        headers[:0] = ["market"]
        headers[:0] = ["date"]
        headers = { idx:val for idx ,val in enumerate(headers)}
    
        data = []
        price_data = data_list[10:]
        for row in price_data:
            if "http" in row:
                continue
            if "Prepared" in row:
                continue
            row = row.split("|")
            row[:0] = ["market"]
            row[:0] = ["date"]
            row = { idx:val for idx ,val in enumerate(row)}
            data.append(row)
        


        clean_header = []
        for key, value in headers.items():
            if key in [0,1,2,4,8,11,12]:
                clean_header.append(value)
        print(clean_header)
        # get the papine data

        papine_data = []
        coronation_data = []
        cross_roads_data = []
        markets = ["papine","coronation","cross_roads"]

        for row in data:
            p_row = []
            cor_row = []
            crss_row = []
        
            for key, value in row.items():
        
                value = value.replace("\n","")
                if key in [2,4,8,11,12]:
                    p_row.append(value)
                    

                if key in [2,4,17,19,20]: 
                    cor_row.append(value)  
                

                if key in [2,4,22,23,25]: 
                    crss_row.append(value)
                

            p_row[:0] = [markets[0]]
            p_row[:0] = [self.date]  

            cor_row[:0]= [markets[1]]
            cor_row[:0]= [self.date]

            crss_row[:0] = [markets[2]]
            crss_row[:0] = [self.date]


            papine_data.extend([p_row])
            coronation_data.extend([cor_row])
            cross_roads_data.extend([crss_row])

        data = []
        data.extend(papine_data)
        data.extend(cross_roads_data)
        data.extend(coronation_data)
        return data

  
    def __get_locations(self,data_list):
        locations = data_list[7].split("|")
        locations = list(filter(lambda loc: len(loc) >3, locations))
        return locations

    def write_to_csv(self,output_directory):
        import pandas as pd
        import os
        output_dir = f"{output_directory}/{self.date}"
        try:
            os.mkdir(output_dir) 
        except OSError as error: 
            print(f"{output_dir} already exists,not recreating")      

        df = pd.DataFrame(self.data)
        df.to_csv(f"{output_dir}/urban_municipal_market_prices_{self.date}.csv",sep="|" ,index=False, header=False)  




In [365]:
#export
def run(input_path:str, output_directory = "."):
    lines = open_file(input_path)
    factory = UrbanMunicipalMarketPricesFactory(lines)        
    factory.write_to_csv(output_directory)
    print("Success :-)")

In [356]:
#export
@call_parse
def main(input_path:Param("The urban municipal prices file path", str),
         output_folder:Param("where the converted file should be written", str),
      ):
    "Clean up the input file and generates individual parish farm gate prices in the specified output folder"
    print(f"input path:{input_path}")
    run(input_path, output_folder)

In [366]:
notebook2script()

Converted 00_municipal_analytics.ipynb.
Converted index.ipynb.
